## Load library modules

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

default_dir = os.path.dirname(os.path.dirname(os.getcwd()))
os.chdir(default_dir)

# (Mac) Uncomment below two lines if there's an error with PyJulia setup (initial execution may take some time)
# from julia.api import Julia
# jl = Julia(compiled_modules=False)

from modWorm import sys_paths
from modWorm import network_params as n_params
from modWorm import network_dynamics as n_dyn
from modWorm import network_interactions as n_inter
from modWorm import network_simulations as n_sim

from modWorm import muscle_body_params as mb_params
from modWorm import muscle_dynamics as m_dyn
from modWorm import body_dynamics as b_dyn
from modWorm import body_simulations as b_sim

from modWorm import animation
from modWorm import utils

### Load experimental connectome data
- Varshney, Lav R., et al. "Structural properties of the Caenorhabditis elegans neuronal network." PLoS computational biology 7.2 (2011): e1001066.
- Haspel, Gal, and Michael J. O'Donovan. "A perimotor framework reveals functional segmentation in the motoneuronal network controlling locomotion in Caenorhabditis elegans.
- Download link: https://www.wormatlas.org/images/NeuronConnect.xls

### Load neurons to muscles mapping data
- WormAtlas, Altun, Z.F., Herndon, L.A., Wolkow, C.A., Crocker, C., Lints, R. and Hall, D.H. (ed.s) 2002-2024.
- Download link: https://www.wormatlas.org/images/NeuronFixedPoints.xls
- If the file is in old .xls format, make sure to save as .xlsx format

In [ ]:
# Construct gap, synaptic connectomes and muscle map from downloaded files
conn_gap, conn_syn = utils.construct_connectome_Varshney(filepath)
muscle_map = utils.construct_muscle_map_Hall(filepath)

## Load pre-defined Nervous System model

In [ ]:
from modWorm import predefined_classes_nv, predefined_classes_mb

celegans_nv = predefined_classes_nv.CelegansWorm_NervousSystem_Julia(conn_gap, conn_syn)          # Define nervous system class
celegans_mb = predefined_classes_mb.CelegansWorm_MuscleBody_Julia(muscle_map)                     # Define muscle + body class
                                                                                                  # Julia versions are used to accelerate simulations

## Define Stimulation

In [ ]:
PLM_neuron_inds = utils.neuron_names_2_inds(['PLML', 'PLMR'])

In [ ]:
simulation_time = 15                                           # Simulation duration in seconds
simulation_steps = int(simulation_time/celegans_nv.timescale)  # Total number of timesteps
                                                               # Note this should be associated with timescale defined above

input_mat = np.zeros((simulation_steps, celegans_nv.network_Size))   # External input for the period of 5 seconds 
input_mat[:, PLM_neuron_inds] = 2000                                 # Inject 2000pA (2nA) into PLML/R (276, 278th neurons)

## Simulate Nervous System

In [ ]:
solution_dict_nv = n_sim.run_network_julia(celegans_nv, input_mat)

## Simulate Biomechanics

In [ ]:
solution_dict_mb = b_sim.run_body_julia(celegans_mb, celegans_nv, solution_dict_nv)

## Analyze results

In [ ]:
print(solution_dict_mb.keys()) # solution_dict has 5 keys
                               # raw_x, raw_y -> original x,y solutions for 24 segments
                               # x, y -> post-processed x,y solutions extrapolated to 192 segments

In [ ]:
# Use post-processed x and y solution (x_solution and y_solution) to visualize the body trajectory
plt.plot(solution_dict_mb['x_solution'][:, 0], solution_dict_mb['y_solution'][:, 0])
plt.ylim(-10, 10)
plt.xlim(-15, 5)

In [ ]:
# Create a video of body dynamics
animation.animate_body(x = solution_dict_mb['x_solution'], y = solution_dict_mb['y_solution'], filename = 'fwd_locomotion',
                       xmin = -50, xmax = 50, ymin = -50, ymax = 50,
                       figsize_x = 10, figsize_y = 10, 
                       background_img_path = False, animation_config = mb_params.CE_animation)